# 0.0 IMPORT

In [42]:
import pandas as pd
import numpy as np
import json
import datetime
import ast
import dtale
import random
import optuna


from sklearn import preprocessing as pp
import pickle

from sklearn import model_selection as ms
import category_encoders as ce

from boruta   import BorutaPy
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.ensemble        import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from xgboost                 import XGBRegressor
from sklearn.linear_model    import LogisticRegression, SGDRegressor
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes     import GaussianNB
from lightgbm                import LGBMRegressor

## 0.1 Function Train and Test

In [2]:
def load_data(url):
    # Opening JSON file
    train = open(url)

    # returns JSON object as a dictionary
    data_train_aux = json.load(train)

    data_train = pd.json_normalize(data_train_aux, record_path = 'data')
    data_train.columns = data_train_aux['columns']
    return data_train

In [3]:
def limpeza_inicial(df):
    
    if 'actual_price' in df.columns:
        
        df = df.loc[df['actual_price'] < 6000]
        
        df = df.drop(columns=['description', 'images','crawled_at','title'])
        prices = df[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()
    
        precos_nulos = df.loc[df['actual_price'].isna(), ['_id','pid']]

        precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')

        df_aux = pd.merge(df, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
        df['actual_price'] = df_aux['actual_price_x'].fillna(0) + df_aux['actual_price_y'].fillna(0)

        df = df.dropna(subset=['actual_price'])
        
    else:
        df = df.drop(columns=['description', 'images','crawled_at','title'])

    return df

In [4]:
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

In [5]:
def junta_dict(dict_list):
        dicionario = {}
        for d in dict_list:
            for key, value in d.items():
                dicionario[key] = value
        return dicionario

In [6]:
def expand_product_detais (df):
    df_pd = df['product_details'].apply(lambda x: f(x))
    
    df_pd = pd.DataFrame([junta_dict(row) for row in df_pd], index = df_pd.index)
    
    df_pd = df_pd.drop(columns=['', ' '])

    # Removendo valores faltantes acima de 50%
    limite_nulos = len(df_pd) * 0.5  
    df_pd = df_pd.dropna(thresh = limite_nulos, axis=1)
    
    return df_pd


In [7]:
def df_pd_plus_df(df, df_pd):
    df = pd.concat([df, df_pd], axis=1)
    df = df.drop( ['product_details'] , axis = 1)
    return df

In [8]:
def ajuste_colunas (df):
    
    df.loc[df['Fabric'].isnull(), 'Fabric'] = 'outros'
    df.loc[df['Pattern'].isnull(), 'Pattern'] = 'outros'
    df.loc[df['Style Code'].isnull(), 'Style Code'] = 'outros'
    df.loc[df['Pack of'].isnull(), 'Pack of'] = 'outros'
    df.loc[df['Type'].isnull(), 'Type'] = 'outros'
    df.loc[df['Sleeve'].isnull(), 'Sleeve'] = 'outros'
    df.loc[df['Fit'].isnull(), 'Fit'] = 'outros'
    df.loc[df['Ideal For'].isnull(), 'Ideal For'] = 'outros'
    df.loc[df['Suitable For'].isnull(), 'Suitable For'] = 'outros'
    df.loc[df['Reversible'].isnull(), 'Reversible'] = 'outros'
    df.loc[df['Fabric Care'].isnull(), 'Fabric Care'] = 'outros'
    
    # # Suponha que você tenha uma coluna chamada 'Fabric' com valores do tipo object
    # df_new['Fabric'] = df_new['Fabric'].astype(str)
    # df_new['Pattern'] = df_new['Pattern'].astype(str)
    # df_new['Style Code'] = df_new['Style Code'].astype(str)
    # df_new['Pack of'] = df_new['Pack of'].astype(str)
    # df_new['Type'] = df_new['Type'].astype(str)
    # df_new['Sleeve'] = df_new['Sleeve'].astype(str)
    # df_new['Fit'] = df_new['Fit'].astype(str)
    # df_new['Ideal For'] = df_new['Ideal For'].astype(str)
    # df_new['Suitable For'] = df_new['Suitable For'].astype(str)
    # df_new['Reversible'] = df_new['Reversible'].astype(str)
    # df_new['Fabric Care'] = df_new['Fabric Care'].astype(str)


    # # Agora você pode usar o método str.split para dividir os valores
    # df_new['Fabric'] = df_new['Fabric'].str.split(',')
    # df_new['Pattern'] = df_new['Pattern'].str.split(',')
    # df_new['Style Code'] = df_new['Style Code'].str.split(',')
    # df_new['Pack of'] = df_new['Pack of'].str.split(',')
    # df_new['Type'] = df_new['Type'].str.split(',')
    # df_new['Sleeve'] = df_new['Sleeve'].str.split(',')
    # df_new['Fit'] = df_new['Fit'].str.split(',')
    # df_new['Ideal For'] = df_new['Ideal For'].str.split(',')
    # df_new['Suitable For'] = df_new['Suitable For'].str.split(',')
    # df_new['Reversible'] = df_new['Reversible'].str.split(',')
    # df_new['Fabric Care'] = df_new['Fabric Care'].str.split(',')
    
    return df

In [9]:
def type_sex(df):
# Dados sem virgulas: Style Code, Pack of, Suitable For, Reversible, 'Ideal For'

# Ideal for: 'Men', 'Men, Boys'/ 'Boys, Men' = 'all_ages',
            # 'Boys, Girls, Men, Women' = 'unisex_all_ages', 'Women, Men' = 'Unisex'
    
    df['Ideal For'] = df['Ideal For'].apply(lambda x : 'men' if x == 'Men'
                                            else 'all_ages' if x == 'Men, Boys'
                                            else 'all_ages' if x == 'Boys, Men'
                                            else 'unisex_all_ages' if x == 'unisex_all_ages'
                                            else 'unisex' if x == 'Women, Men'
                                            else 'outros')
    
    # Novas Features de roupa

    df['cotton'] = df['Fabric'].apply(lambda x: 1 if 'cotton' in x.lower() else 0)
    df['poly'] = df['Fabric'].apply(lambda x: 1 if 'poly' in x.lower() else 0)
    df['not_cotton_or_poly'] = df['Fabric'].apply(lambda x: 1 if ('poly' not in x.lower()) and ('cotton' not in x.lower()) else 0)
    
    return df

In [10]:
def feature_engine (df):
    
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    return df


In [11]:
def split_df(df):
    
    if 'actual_price' in df.columns:
        X = df.drop(columns=['_id','pid','actual_price'], axis=1)
        y = df['actual_price']
        X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.20)
        data_train = pd.concat([X_train, y_train], axis=1)
        data_test = pd.concat([X_test, y_test], axis=1)
        
        return X_train, X_test, y_train, y_test
    else:
        X_prod = df.drop(columns=['_id','pid'], axis=1)
        return X_prod 


In [12]:
def var_resposta(y_train):
    # Tranformando a variável resposta em logaritmica
    y_train = np.log1p(y_train)
    return y_train

In [13]:
def rescaling_train(df_train):
    
    # 'avg_delivery_time_days',
    mm = pp.MinMaxScaler()

    df_train['avg_delivery_time_days'] = mm.fit_transform(df_train[['avg_delivery_time_days']])
    pickle.dump(mm, open('parameters/mm_avg_delivery_time_days.pkl','wb'))

    rs_average_rating = pp.RobustScaler()
    rs_number_of_reviews = pp.RobustScaler()

    df_train['average_rating'] = rs_average_rating.fit_transform(df_train[['average_rating']])
    pickle.dump(rs_average_rating, open('parameters/rs_average_rating.pkl','wb'))

    df_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(df_train[['number_of_reviews']])
    pickle.dump(rs_number_of_reviews, open('parameters/rs_number_of_reviews.pkl','wb'))
    
    return df_train

    ##### Fazer Robust Scaler da variável resposta (??) #####

def rescaling_test(df_test):
    
    mm = pickle.load(open('parameters/mm_avg_delivery_time_days.pkl','rb'))
    df_test['avg_delivery_time_days'] = mm.transform(df_test[['avg_delivery_time_days']])
    
    rs_ar = pickle.load(open('parameters/rs_average_rating.pkl','rb'))
    df_test['average_rating'] = rs_ar.transform(df_test[['average_rating']])
    
    rs_nr = pickle.load(open('parameters/rs_number_of_reviews.pkl','rb'))
    df_test['number_of_reviews'] = rs_nr.transform(df_test[['number_of_reviews']])
    
    return df_test

In [14]:
def tranform_train(X_train, y_train):
    
    
#    #ONE HOT ENCODE
#    X_train = pd.get_dummies( X_train, prefix = ['IF'], columns = ['Ideal For'] )
#    X_train = pd.get_dummies( X_train, prefix = ['RV'], columns = ['Reversible'] )
#    X_train = pd.get_dummies( X_train, prefix = ['SF'], columns = ['Suitable For'] )
#    X_train = pd.get_dummies( X_train, prefix = ['PO'], columns = ['Pack of'] )
#    X_train = pd.get_dummies( X_train, prefix = ['PT'], columns = ['Pattern'] )
#    
    #FREQUENCY ENCODER
    for att in ['brand',
                'product',
                'seller',
                'category',
                'Suitable For',
                'Reversible',
                'Ideal For',
                'out_of_stock',
                'Pattern',
                'Pack of'
                ]:
        fe =  ce.CountEncoder(cols=[att], normalize=True).fit(X_train)
        pickle.dump(fe,open(f'parameters/fe_{att}.pkl', 'wb'))
        X_train = fe.transform(X_train)
    
    # Instancie o codificador alvo para 'Style Code'
    te_style_code = ce.TargetEncoder()
    te_pack_of = ce.TargetEncoder()
    
    X_train['Style Code'] = te_style_code.fit_transform(X_train['Style Code'], y_train) 
    pickle.dump(te_style_code, open('parameters/te_style_code.pkl', 'wb'))

    # Instancie o codificador alvo para 'Pack of'
    X_train['Pack of'] = te_pack_of.fit_transform(X_train['Pack of'], y_train) 
    pickle.dump(te_pack_of, open('parameters/te_pack_of.pkl', 'wb'))
    
    return X_train

def tranform_test(X_test):
    
    #ONE HOT ENCODE
#    X_test = pd.get_dummies( X_test, prefix = ['IF'], columns = ['Ideal For'] )
#    X_test = pd.get_dummies( X_test, prefix = ['RV'], columns = ['Reversible'] )
#    X_test = pd.get_dummies( X_test, prefix = ['SF'], columns = ['Suitable For'] )
#    X_test = pd.get_dummies( X_test, prefix = ['PO'], columns = ['Pack of'] )
#    X_test = pd.get_dummies( X_test, prefix = ['PT'], columns = ['Pattern'] )
    
    # FREQUENCY ENCODER
    for att in ['brand',
                'product',
                'seller',
                'category',
                'Suitable For',
                'Reversible',
                'Ideal For',
                'out_of_stock',
                'Pattern',
                'Pack of'
                ]:
        fe = pickle.load(open(f'parameters/fe_{att}.pkl', 'rb'))
        X_test = fe.transform(X_test)

    # TARGET ENCODE
    te_style_code = pickle.load(open('parameters/te_style_code.pkl','rb'))
    X_test['Style Code'] = te_style_code.transform(X_test['Style Code'])

    te_pack_of = pickle.load(open('parameters/te_pack_of.pkl','rb'))
    X_test['Pack of'] = te_pack_of.transform(X_test['Pack of'])

    return X_test

In [15]:
def feature_select_LGBM (X_train, y_train):
    
    # Crie e treine o modelo LGBMClassifier
    lgb_model = LGBMRegressor(n_jobs=-1, random_state=42)
    lgb_model.fit(X_train, y_train)
    # Obtenha as importâncias das características do modelo
    importances = lgb_model.feature_importances_

    # Ordene as características por importância decrescente
    indices = np.argsort(importances)[::-1]

    # Imprima o ranking das características
    print('Feature ranking')

    df = pd.DataFrame()
    for i, j in zip(X_train.columns, importances):
        aux = pd.DataFrame({'feature': i, 'importance': j}, index=[0])

        df = pd.concat([df, aux], axis=0)
        df = df.sort_values('importance', ascending=False)
    return df

In [16]:
def feature_selected_BORUTA(X_train, y_train):
    rf = RandomForestRegressor()
    
    # Define Boruta - Essa peste demora 1 hora ou mais.
    boruta = BorutaPy( rf, n_estimators = 'auto', verbose=2, random_state=42).fit( X_train, y_train)

    cols_selected = boruta.support_.tolist()
    print(cols_selected)

In [17]:
def feature_selected_manual (df):
    features_selected = [
        'average_rating',
        #'number_of_reviews',
        'brand',
        # 'category',
        # 'crawled_at',
        'out_of_stock',
        #'avg_delivery_time_days',
        #  'product_details',
        'seller',
        #  'sub_category',
        #  'fabrication_time',
        #  'title',
        #  'Fabric',
        #  'Pattern',
        'cotton',
        'poly',
        'Style Code',
        'Pack of',
        #  'Type',
        #  'Sleeve',
        #  'Fit',
        'Ideal For',
        'Suitable For',
        'Reversible',
        #  'Fabric Care',
        'product'
        ]
    return df[features_selected]

In [37]:
def MachineLearning(X_train, y_train, X_test, y_test):
    SEED = 42

    lista_de_medidas = ['SMAPE']
    nome_das_medidas = ['SMAPE']

    lista_de_modelos = [XGBRegressor(learning_rate=0.1, n_estimators=100, max_depth=3),
                        SGDRegressor(random_state=SEED), 
                        LGBMRegressor(random_state=SEED, n_jobs=-1, force_row_wise=True, ),
                        DecisionTreeRegressor(random_state=SEED),
                        KNeighborsRegressor(n_neighbors=5,  weights='distance',n_jobs=-1),
                        BaggingRegressor(),
                        RandomForestRegressor(random_state=SEED)]

    nome_dos_modelos = ['XGBoost', 
                        'SGD', 
                        'LGBM', 
                        'DecisionTree',
                        'KNN',
                        'Bagging',
                        'RandomForest']

    resultados0 = {}
    



    for i in range(len(lista_de_modelos)):
        print('Rodando modelo: ' + nome_dos_modelos[i])
        
        model = lista_de_modelos[i]
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        print(y_pred)
        
        smape = 100 / len(y_test) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test)) / (np.abs(np.expm1(y_test)) + np.abs(np.expm1(y_pred))))
        # Calculo meigarom: 100 / len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))

        resultados0[nome_dos_modelos[i]] = [smape]
    
    resultados = pd.DataFrame(resultados0, index = nome_das_medidas).T

    return resultados

In [76]:
def cross_validation (model_name, X_training, Y_training, kfold, model, verbose = False ):
    smape_list = []
    
    
    # Star an end date validation
    for k in reversed( range (1, kfold+1) ):
        if verbose:
            print('\nKfold: {}'.format(k))
        teste_size = (100/(k))/100
        if teste_size < 1:
            print(teste_size)
            X_trein, X_test, y_trein, y_test = train_test_split(X_training, Y_training, test_size=teste_size, random_state=42)
        else:
            pass

        # X_treino = X_training.loc[indices_amostra]
        # print(f'X_treino: {X_treino}')
        # Y_treino = Y_training.loc[indices_amostra]
        # print(f'Y_treino: {Y_treino}')
        
        # X_test = X_training.drop(index=X_treino.index)
        # print(f'X_test: {X_test}')
        # Y_test = Y_training.drop(index=Y_treino.index)
        # print(f'Y_test: {Y_test}')

        model.fit(X_trein, y_trein)
        
        y_pred = model.predict(X_test)
        
        smape = 100 / len(y_test) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test)) / (np.abs(np.expm1(y_test)) + np.abs(np.expm1(y_pred))))
        # Calculo meigarom: 100 / len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))
        
        # Guardando as performaces
        smape_list.append( smape )
        
        
    resposta_modelo = pd.DataFrame( {'Model Name': model_name,
                                     'SMAPE': smape_list},
                                   index = [0])
    
    return resposta_modelo #, curiosidade

In [47]:
def fine_tunnung (X_train, y_train): 
    #RANDON SEARCH
    param = {'n_estimators' : [1500, 1700, 3000, 3500],
         'eta' : [0.01, 0.03],
         'max_depth' : [3, 5, 9],
         'subsample': [0.1, 0.5, 0.7],
         'colsample_bytree' : [0.3, 0.7, 0.9],
         'min_child_weight' : [3, 8, 15]}

    MAX_EVAL = 10
    
    final_result = pd.DataFrame()
    for i in range (MAX_EVAL):
        # Escolher os parâmetros aleatórios
        hp = { k: random.sample(v, 1)[0] for k, v in param.items() }
        print(hp)


        # model
        model_xgb = XGBRegressor(objective = 'reg:squarederror',
                                n_estimators=hp['n_estimators'],
                                eta = hp['eta'],
                                max_depth = hp['max_depth'],
                                subsample = hp['subsample'],
                                colsample_bytree = hp['colsample_bytree'],
                                min_child_weight = hp['min_child_weight'])
        # performance

        result = cross_validation (model_name = 'XGBRegressor',
                                   X_training = X_train,
                                   Y_training = y_train,
                                   kfold = 5,
                                   model = model_xgb,
                                   verbose = True )
        final_result = pd.concat([final_result, result])
    final_result

In [21]:
def tratamento_inicial (df):
    
    df = limpeza_inicial (df)
    df_pd = expand_product_detais (df)
    df = df_pd_plus_df(df, df_pd)
    df = ajuste_colunas (df)
    df = type_sex(df)
    df = feature_engine (df)
    
    return df

In [22]:
def best_model(model, x_test):
    
    yhat = np.expm1(model.predict(x_test))
    
    return yhat

## 0.3 Function Subission CSV

In [23]:
def submission(df_sub,yhat):
    # Preparando os dados para a submissão

    df_submission = pd.DataFrame()
    df_submission['pid'] = df_sub['pid']
    df_submission['actual_price'] = yhat

    # Exportando como CSV
    df_submission.to_csv('submissions/submission_g01.csv', index=False)

## 0.4 Load Data

In [24]:
url_treino = '../2023-10-20-Hackday_6/dataset/raw/train.json'
url_prod = '../2023-10-20-Hackday_6/dataset/raw/test.json'

df_treino = load_data(url_treino)
df_prod = load_data(url_prod)

# 1.0 Run Train Test

In [25]:
df_treino = tratamento_inicial(df_treino)

In [26]:
df_prod = tratamento_inicial(df_prod)

In [27]:
X_train, X_test, y_train, y_test = split_df(df_treino)
y_train = var_resposta(y_train)
y_test = var_resposta(y_test)

In [28]:
X_prod = split_df(df_prod)

In [29]:
X_train = rescaling_train(X_train)
X_test = rescaling_test(X_test)
X_prod = rescaling_test(X_prod)

In [ ]:
X_train = tranform_train(X_train, y_train)
X_test = tranform_test(X_test)
X_prod = tranform_test(X_prod)

In [31]:
X_train = feature_selected_manual (X_train)
X_test = feature_selected_manual (X_test)
X_prod = feature_selected_manual (X_prod)

In [ ]:
fetures = feature_select_LGBM (X_train, y_train)
fetures

In [33]:
#feature_selected_BORUTA(X_train, y_train)

In [34]:
X_train = feature_selected_manual (X_train)
X_test = feature_selected_manual (X_test)
X_prod = feature_selected_manual (X_prod)

In [ ]:
resultado = MachineLearning(X_train, y_train, X_test, y_test)
resultado

In [ ]:
fine_tunnung (X_train, y_train)

# 2.0 Submission Preparation

In [ ]:
model = SGDRegressor(random_state=42)
model_train = model.fit(X_train, y_train)

In [ ]:
yhat = best_model(model_train, X_prod)

# 3.0 Submission CSV

In [ ]:
# Preparando os dados para a submissão
df_submission = pd.DataFrame()
df_submission['pid'] = df_prod['pid']
df_submission['actual_price'] = yhat
# Exportando como CSV
df_submission.to_csv('submissions/submission_g02.csv', index=False)